In [43]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


In [44]:
import pandas as pd
import numpy as np


In [45]:
df = pd.read_csv('fake_news_dataset/train.csv')

In [46]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [47]:
df.shape

(20800, 5)

In [48]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [49]:
df = df.dropna()

In [50]:
X = df.drop('label',  axis = 1)
Y = df['label']

In [51]:
print(X.shape,Y.shape)

(18285, 4) (18285,)


In [52]:
sen = X.copy()

sen.reset_index(inplace = True)

In [53]:
import nltk
import re
from nltk.tokenize import sent_tokenize ,word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [54]:
stop_word=stopwords.words("english")
stop_word.extend(['from', 'subject', 're', 'edu', 'use'])
ps = PorterStemmer()
list = []
for i in range(0, len(sen)):
    message = re.sub('[^a-zA-Z]', ' ', sen['title'][i])
    message = message.lower()
    message = message.split()
    
    message = [ps.stem(word) for word in message if not word in stop_word]
    message = ' '.join(message)
    list.append(message)

In [55]:
list

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [56]:
# One hot representation
### Vocabulary size
voc_size=5000
one_hot_rep = [one_hot(words,voc_size)for words in list]
one_hot_rep

[[4084, 2945, 18, 1092, 861, 2995, 1170, 1855, 1036, 2838],
 [1128, 907, 1335, 1682, 4074, 4370, 670],
 [1858, 2592, 700, 801],
 [3914, 3650, 3975, 3942, 3979, 1545],
 [3659, 4074, 1574, 3063, 2729, 2842, 4074, 3239, 2861, 3543],
 [4705,
  4717,
  2452,
  1349,
  822,
  4306,
  486,
  352,
  363,
  4466,
  4389,
  1452,
  2154,
  1081,
  670],
 [2656, 2436, 662, 3020, 1173, 1949, 3183, 504, 241, 4632, 4573],
 [2509, 2371, 2527, 2140, 912, 2223, 4306, 4984, 241, 4632, 4573],
 [4067, 1844, 1764, 2554, 4288, 1249, 2902, 4444, 4306, 4847],
 [4498, 1012, 3978, 2017, 3959, 1414, 206, 2437],
 [2757, 4585, 938, 4987, 4191, 4471, 3967, 4365, 3673, 221, 4582],
 [3942, 3049, 861, 1249, 4306, 912],
 [2227, 650, 240, 3794, 4652, 930, 4003, 2295, 619],
 [2360, 2468, 2160, 1067, 4407, 4884, 4184, 241, 4632, 4573],
 [271, 677, 3723, 3825, 3241, 241, 4632, 4573],
 [4503, 2329, 4543, 4198, 2238, 3666, 1927, 3147, 4048, 1922],
 [3805, 2292, 907],
 [1267, 945, 3229, 1591, 4306, 4365, 744, 670],
 [161, 336

In [57]:
# All sentences length are different . So to make it fixed length we use padding
sent_length = 20
embedded_docs = pad_sequences(one_hot_rep,padding = 'pre', maxlen = sent_length) #input to embedding layer
print(embedded_docs)

[[   0    0    0 ... 1855 1036 2838]
 [   0    0    0 ... 4074 4370  670]
 [   0    0    0 ... 2592  700  801]
 ...
 [   0    0    0 ...  241 4632 4573]
 [   0    0    0 ...  568 1518   95]
 [   0    0    0 ... 1825 1863 3527]]


In [58]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1128,  907, 1335, 1682, 4074, 4370,  670])

In [59]:
# Creating Model
emb_dim = 40
model = Sequential()
model.add(Embedding(voc_size,emb_dim,input_length = sent_length))
model.add(LSTM(100)) # 1 Lstm layer having 100 neurons
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
X_final = np.array(embedded_docs)
Y_final = np.array(Y)
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

In [62]:
model.fit(X_train,Y_train,validation_data = (X_test,Y_test),epochs = 10,batch_size = 64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 13s 1ms/sample - loss: 0.3605 - acc: 0.8243 - val_loss: 0.1980 - val_acc: 0.9132
Epoch 2/10
12250/12250 [==============================] - 10s 839us/sample - loss: 0.1476 - acc: 0.9416 - val_loss: 0.1975 - val_acc: 0.9173
Epoch 3/10
12250/12250 [==============================] - 10s 850us/sample - loss: 0.1013 - acc: 0.9631 - val_loss: 0.2210 - val_acc: 0.9147
Epoch 4/10
12250/12250 [==============================] - 11s 871us/sample - loss: 0.0751 - acc: 0.9736 - val_loss: 0.2962 - val_acc: 0.9135
Epoch 5/10
12250/12250 [==============================] - 11s 889us/sample - loss: 0.0536 - acc: 0.9831 - val_loss: 0.2881 - val_acc: 0.9105
Epoch 6/10
12250/12250 [==============================] - 11s 917us/sample - loss: 0.0384 - acc: 0.9872 - val_loss: 0.3727 - val_acc: 0.9084
Epoch 7/10
12250/12250 [==============================] - 11s 896us/sample - loss: 0.0244 - acc: 0.9927 - v

In [67]:
y_pred = model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [68]:
confusion_matrix(Y_test,y_pred)

array([[3081,  338],
       [ 214, 2402]], dtype=int64)

In [69]:
accuracy_score(Y_test,y_pred)

0.9085335542667772